# Building the GBDT here

##Run small data set locally (2-Trees)

In [1]:
##Let us write spark code here for page rank
import os
import sys

spark_home = os.environ['SPARK_HOME'] = '/Users/maktrix/Dropbox/Berkeley/W261_ML_scale/spark-1.5.1-bin-hadoop2.6'
# spark_home = os.environ['SPARK_HOME'] = '/usr/local/Cellar/apache-spark/1.5.0/libexec/'
if not spark_home:
    raise ValueError('SPARK_HOME enviroment variable is not set')
sys.path.insert(0,os.path.join(spark_home,'python'))
sys.path.insert(0,os.path.join(spark_home,'python/lib/py4j-0.8.2.1-src.zip'))
execfile(os.path.join(spark_home,'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.5.1
      /_/

Using Python version 2.7.10 (default, Oct 19 2015 18:31:17)
SparkContext available as sc, HiveContext available as sqlContext.


In [39]:
%%writefile GBDT_Model.py
#!/usr/bin/env python
import sys
from pyspark import SparkContext
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.util import MLUtils
from types import *

def parsePoint(point):
    ##exclude the first 13 features always as they are numerical
    return [(i,j)for i,j in enumerate(point.split(',')[14:])]

def createOneHotDict(inputData):
    sampleDistinctFeats = (inputData #.flatMap(filterNumbers)
                         .flatMap(lambda x: x)
                      .distinct())
    return (sampleDistinctFeats
                           .zipWithIndex()
                             .collectAsMap())  

# TODO: Replace <FILL IN> with appropriate code
def oneHotEncoding(rawFeats, OHEDict, numOHEFeats,first13):
    rawFeats=sorted(rawFeats)
    num=[]
    arr=[]
    for j,i in enumerate(first13):
        if i != '':
            num.append(j)
            arr.append(i)
    OHEFeats=[]
    for i in OHEDict.keys():
        if i in rawFeats:
            OHEFeats.append(i)
                  
    return SparseVector(26+13,sorted(num+\
                            [OHEDict[i]+13 for i in OHEFeats]),\
                        arr+[1.0 for i in range(len(OHEFeats))]) #[i for i in first13]+

def parseOHEPoint(point, OHEDict, numOHEFeats):
    featurelist=point.strip().split(',')
    return LabeledPoint(featurelist[0],oneHotEncoding([(i,j) for i,j in enumerate(featurelist[14:]) ],\
                                                      OHEDict, 26+13,[i for i in featurelist[1:14]]))

def maaro(z):
    if z[1]=='':
        return (-1,0)
    else:
        return(z,1)

    
#main GBDT function
def run_GBDT(input_file,output_file,iterations):
    dataRDD=sc.textFile(input_file).map(lambda x: x.replace('\t',','))
    #Now let us create labeled point from data
    dataRDDParsed=dataRDD.map(parsePoint).cache()
    featSet=dataRDDParsed.flatMap(lambda x: x).map(maaro).reduceByKey(lambda a,b: a+b).takeOrdered(26,lambda (k,v): -v)
    #reduceByKey(lambda x,y:x+y).takeOrdered(25,lambda (k,v):-v)
    #print featSet
    #OHEdict=createOneHotDict(dataRDDParsed,featSet)
    OHEdict={}
    for i,x in enumerate(featSet):
#         print i,x
        OHEdict[x[0]]=i
   
    #print oneHotEncoding(dataRDDParsed,OHEdict,numSampleOHEFeats,)
    #Now let us create a dictionary of points
#     weights=[.8,.1,.1]
#     seed=42
#     trainRDD,validateRDD,testRDD=dataRDD.randomSplit(weights,seed)
#     OHETrainData = trainRDD.map(lambda point: parseOHEPoint(point, OHEdict, 39))
    OHETrainData = dataRDD.map(lambda point: parseOHEPoint(point, OHEdict, 39))
#     print OHETrainData.take(1)
#     print OHETrainData.count()

    model = (GradientBoostedTrees.trainClassifier(OHETrainData, loss = 'logLoss', numIterations=2, 
             categoricalFeaturesInfo={}, learningRate = 0.1, maxDepth = 7, maxBins = 2))
    
    sc.parallelize([model.toDebugString()]).coalesce(1).saveAsTextFile(output_file)  
    
#Execute code    
if __name__== '__main__':
    if len(sys.argv) < 3:
        print >> sys.stderr, "Usage: page_rank <initialize sc ?> <input file> <output file> <damping factor> <number_of_iterations>"
        exit(-1)

    input_file = sys.argv[1]
    output_file = sys.argv[2]
    input_iterations =int(sys.argv[3])
    
    sc = SparkContext(appName="GBDT")
    run_GBDT(input_file,output_file,input_iterations)
    sc.stop()


Overwriting GBDT_Model.py


In [40]:
#Run first Spark job to generate decision tree rules
!rm -r gbdt_model_output_local
!time /Users/maktrix/Dropbox/Berkeley/W261_ML_scale/spark-1.5.1-bin-hadoop2.6/bin/spark-submit ./GBDT_Model.py dac_sample.txt gbdt_model_output_local 2
!cat gbdt_model_output_local/part-00000

rm: gbdt_model_output: No such file or directory
15/12/14 19:50:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/12/14 19:50:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
15/12/14 19:50:24 WARN MetricsSystem: Using default name DAGScheduler for source because spark.app.id is not set.

real	0m41.129s
user	0m28.034s
sys	0m1.549s


In [26]:
%%writefile Convert_Python_Rules.py
#!/usr/bin/env python
import sys

def rules_python(input_file, output_file):
    
    spacing = '    '
    
    with open(input_file,'r') as f_in:
        with open(output_file,'w+') as f_out:
            all_lines = []

            for line in f_in:
                # print(model.toDebugString())
    #             model.txt = model.toDebugString().split('\n')

    #             for line in model.txt:
                text = line.split()
                if text == []:
                    pass
                else:
                    all_lines.append(text)

            starting_indent = 1
            if_indents = {0: starting_indent}
            if_num = 0

            for i in range(len(all_lines)):
                if all_lines[i][0] == 'Tree':
                    indents = starting_indent - 1
                    bin_num = 0
#                     tabs = '\t'*starting_indent
                    tabs = spacing*starting_indent
                    tree_num = int(all_lines[i][1].strip(':'))
#                     print tabs + '# Tree ' + str(tree_num)
                    f_out.write(tabs + '# Tree ' + str(tree_num)+'\n')

                elif all_lines[i][0] == 'If':
                    indents = indents + 1
                    if_indents[if_num] = indents
#                     tabs = '\t'*indents
                    tabs = spacing*indents
#                     print tabs + 'if ' + all_lines[i][1] + '[' + all_lines[i][2] + ']' + all_lines[i][3] + all_lines[i][4] + ':'
                    f_out.write(tabs + 'if ' + all_lines[i][1] + '[' + all_lines[i][2] + ']' + all_lines[i][3] + all_lines[i][4] + ':'+'\n')
                    if_num += 1

                elif all_lines[i][0] == 'Else':
                    indents = if_indents[if_num - 1]
#                     tabs = '\t'*indents
                    tabs = spacing*indents
#                     print tabs + 'else:'
                    f_out.write(tabs + 'else:'+'\n')
                    if_num -= 1

                elif all_lines[i][0] == 'Predict:':
                    indents = indents + 1
#                     tabs = '\t'*indents
                    tabs = spacing*indents
#                     print tabs + 'bin_num[' + str(tree_num) + '] = ' + str(bin_num)
                    f_out.write(tabs + 'bin_num[' + str(tree_num) + '] = ' + str(bin_num)+'\n')
                    bin_num += 1
                    indents = indents - 1

if __name__== '__main__':
    rules_python(sys.argv[1],sys.argv[2])
    

Overwriting Convert_Python_Rules.py


In [36]:
%%writefile GBDT2_Bins.py
#!/usr/bin/env python
import sys
from pyspark import SparkContext
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.util import MLUtils
from types import *

def get_bins(feature):
    bin_num = {} 
    
#NEW_RULES_HERE
    
    return bin_num

def parsePoint(point):
    ##exclude the first 13 features always as they are numerical
    return [(i,j)for i,j in enumerate(point.split(',')[14:])]

def createOneHotDict(inputData):
    sampleDistinctFeats = (inputData #.flatMap(filterNumbers)
                         .flatMap(lambda x: x)
                      .distinct())
    return (sampleDistinctFeats
                           .zipWithIndex()
                             .collectAsMap())  

# TODO: Replace <FILL IN> with appropriate code
def oneHotEncoding(rawFeats, OHEDict, numOHEFeats,first13):
    rawFeats=sorted(rawFeats)
    num=[]
    arr=[]
    for j,i in enumerate(first13):
        if i != '':
            num.append(j)
            arr.append(i)
    OHEFeats=[]
    for i in OHEDict.keys():
        if i in rawFeats:
            OHEFeats.append(i)
                  
    return SparseVector(26+13,sorted(num+\
                            [OHEDict[i]+13 for i in OHEFeats]),\
                        arr+[1.0 for i in range(len(OHEFeats))]) #[i for i in first13]+

def parseOHEPoint(point, OHEDict, numOHEFeats):
    featurelist=point.strip().split(',')
    return LabeledPoint(featurelist[0],oneHotEncoding([(i,j) for i,j in enumerate(featurelist[14:]) ],\
                                                      OHEDict, 26+13,[i for i in featurelist[1:14]]))

def maaro(z):
    if z[1]=='':
        return (-1,0)
    else:
        return(z,1)

def run_bins(input_file,output_file):
    #repeat of before to generate OHETrainDataBins
    dataRDD=sc.textFile(input_file).map(lambda x: x.replace('\t',','))
    dataRDDParsed=dataRDD.map(parsePoint).cache()
    featSet=dataRDDParsed.flatMap(lambda x: x).map(maaro).reduceByKey(lambda a,b: a+b).takeOrdered(26,lambda (k,v): -v)
    OHEdict={}
    for i,x in enumerate(featSet):
        OHEdict[x[0]]=i
    OHETrainData = dataRDD.map(lambda point: parseOHEPoint(point, OHEdict, 39))
    
    #apply the new decision tree
    OHETrainDataDense = OHETrainData.map(lambda x: SparseVector.toArray(x.features))
    OHETrainDataBins = OHETrainDataDense.map(lambda x: get_bins(x))
    OHETrainDataBins.coalesce(1).saveAsTextFile(output_file)


if __name__== '__main__':
    
    if len(sys.argv) < 2:
        print >> sys.stderr, "Usage: page_rank <initialize sc ?> <input file> <output file> <damping factor> <number_of_iterations>"
        exit(-1)

    input_file = sys.argv[1]
    output_file = sys.argv[2]
    
    sc = SparkContext(appName="GBDT2")
    run_bins(input_file,output_file)
    sc.stop()



Overwriting GBDT2_Bins.py


In [37]:
#KEY STEP: insert the decision tree rules into the Python file
#ALWAYS DO THIS STEP LOCALLY (DON'T TRY TO DO THIS IN THE CLOUD)
!python ./Convert_Python_Rules.py gbdt_model_output_local/part-00000 gbdt_python_rules.txt
!sed -i '' '/#NEW_RULES_HERE/r gbdt_python_rules.txt' GBDT2_Bins.py

In [38]:
#Run second Spark job to place data into bins
!rm -r gbdt_bins_output_local
!time /Users/maktrix/Dropbox/Berkeley/W261_ML_scale/spark-1.5.1-bin-hadoop2.6/bin/spark-submit ./GBDT2_Bins.py dac_sample.txt gbdt_bins_output_local
!cat gbdt_bins_output_local/part-00000

15/12/14 21:59:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/12/14 21:59:54 WARN MetricsSystem: Using default name DAGScheduler for source because spark.app.id is not set.

real	0m38.666s
user	0m14.376s
sys	0m1.243s
{0: 75, 1: 81}
{0: 85, 1: 24}
{0: 112, 1: 112}
{0: 2, 1: 15}
{0: 72, 1: 63}
{0: 2, 1: 16}
{0: 2, 1: 16}
{0: 66, 1: 15}
{0: 31, 1: 118}
{0: 5, 1: 15}
{0: 0, 1: 5}
{0: 18, 1: 27}
{0: 38, 1: 63}
{0: 17, 1: 25}
{0: 17, 1: 25}
{0: 18, 1: 17}
{0: 67, 1: 16}
{0: 13, 1: 107}
{0: 12, 1: 75}
{0: 100, 1: 63}
{0: 60, 1: 117}
{0: 18, 1: 28}
{0: 20, 1: 25}
{0: 17, 1: 48}
{0: 3, 1: 54}
{0: 4, 1: 25}
{0: 63, 1: 125}
{0: 51, 1: 15}
{0: 60, 1: 118}
{0: 101, 1: 111}
{0: 13, 1: 95}
{0: 3, 1: 47}
{0: 74, 1: 67}
{0: 85, 1: 71}
{0: 74, 1: 63}
{0: 56, 1: 95}
{0: 4, 1: 21}
{0: 118, 1: 103}
{0: 16, 1: 53}
{0: 122, 1: 123}
{0: 60, 1: 117}
{0: 88, 1: 67}
{0: 53, 1: 112}
{0: 1, 1: 8}
{0: 2, 1: 15}
{0: 4, 1: 59}
{0: 60,

##Run small data set on EMR (2-Trees)

In [58]:
#copy data file to S3
!aws s3 cp dac_sample.txt s3://hw14/14_4_GBDT/dac_sample.txt

upload: ./dac_sample.txt to s3://hw14/14_4_GBDT/dac_sample.txt


In [42]:
%%writefile emr_config_spark_max.json
[
  {
    "Classification": "spark",
    "Properties": {
      "maximizeResourceAllocation": "true"
    }
  }
]


Writing emr_config_spark_max.json


In [43]:
#create cluster
!aws emr create-cluster --name "14_4_GBDT_test" --release-label emr-4.2.0 --applications Name=Spark --instance-count 3 --use-default-roles --ec2-attributes KeyName=w261_key2 --instance-type m3.xlarge  --enable-debugging --log-uri s3://hw14/14_4_GBDT --configurations file://./emr_config_spark_max.json

j-QI25Z1XCDVBK


In [59]:
#upload pyspark script to cluster
!scp -i /Users/maktrix/Dropbox/Berkeley/W261_ML_scale/w261_key2.pem GBDT_Model.py hadoop@ec2-52-35-71-1.us-west-2.compute.amazonaws.com:.

#delete output folder with same name
!aws s3 rm --recursive s3://hw14/14_4_GBDT/gbdt_model_output_emr

GBDT_Model.py                                 100% 3675     3.6KB/s   00:00    


#### Run 1st Spark job on EMR by ssh into the cluster and submit spark job with below commands:
- ssh -i /Users/maktrix/Dropbox/Berkeley/W261_ML_scale/w261_key2.pem hadoop@ec2-52-35-71-1.us-west-2.compute.amazonaws.com
- time /usr/lib/spark/bin/spark-submit --master yarn-cluster ./GBDT_Model.py s3n://hw14/14_4_GBDT/dac_sample.txt s3n://hw14/14_4_GBDT/gbdt_model_output_emr 2

In [60]:
#KEY STEP: insert the decision tree rules into the Python file
#ALWAYS DO THIS STEP LOCALLY (DON'T TRY TO DO THIS IN THE CLOUD)
!rm -r gbdt_model_output_emr
!aws s3 cp --recursive s3://hw14/14_4_GBDT/gbdt_model_output_emr gbdt_model_output_emr
!python ./Convert_Python_Rules.py gbdt_model_output_emr/part-00000 gbdt_python_rules.txt
!sed -i '' '/#NEW_RULES_HERE/r gbdt_python_rules.txt' GBDT2_Bins.py

rm: gbdt_model_output_emr: No such file or directory
download: s3://hw14/14_4_GBDT/gbdt_model_output_emr/_SUCCESS to gbdt_model_output_emr/_SUCCESS
download: s3://hw14/14_4_GBDT/gbdt_model_output_emr/part-00000 to gbdt_model_output_emr/part-00000


In [61]:
#upload second pyspark script to cluster
!scp -i /Users/maktrix/Dropbox/Berkeley/W261_ML_scale/w261_key2.pem GBDT2_Bins.py hadoop@ec2-52-35-71-1.us-west-2.compute.amazonaws.com:.

#delete output folder with same name
!aws s3 rm --recursive s3://hw14/14_4_GBDT/gbdt_bins_output_emr

GBDT2_Bins.py                                 100%  220KB 220.4KB/s   00:00    


#### Run 2nd Spark job on EMR by ssh into the cluster and submit spark job with below commands:
- ssh -i /Users/maktrix/Dropbox/Berkeley/W261_ML_scale/w261_key2.pem hadoop@ec2-52-35-71-1.us-west-2.compute.amazonaws.com
- time /usr/lib/spark/bin/spark-submit --master yarn-cluster ./GBDT2_Bins.py s3n://hw14/14_4_GBDT/dac_sample.txt s3n://hw14/14_4_GBDT/gbdt_bins_output_emr

In [62]:
#get the bins results
!rm -r gbdt_bins_output_emr
!aws s3 cp --recursive s3://14/hw14_4_GBDT/gbdt_bins_output_emr gbdt_bins_output_emr
!cat gbdt_bins_output_emr/part-00000

rm: gbdt_bins_output_emr: No such file or directory
cat: gbdt_bins_output_emr/part-00000: No such file or directory
